## Analyse théorique

1. Quel est le nombre moyen d’utilisateurs actifs à chaque slot dans la cellule ? <br>
**Réponse:** <br>
\begin{equation}
    p\lambda\pi R^2.
\end{equation}
<br>
2. Calculer $N_{max}$. <br>
**Réponse:** <br>
\begin{equation}
    N_{max} = \left\lceil \frac{C}{W\log_2\left(1+\frac{K}{R^\gamma}\right)}\right\rceil.
\end{equation}
<br>
3. La nature de l'ensemble $A_k$. <br>
**Réponse:** <br>
C'est l'anneaxu $A(R_{k,min},\,R_{k,max})$, où 
\begin{equation}
    R_{k,min}=
    \begin{cases}
        0& k=1\\
        R_{k-1,max}& k\geq 2.
    \end{cases}
\end{equation}
\begin{equation}
    R_{k,max}=\min\left(\left(\frac{K}{2^{\frac{C}{kW}}-1}\right)^{1/\gamma},\,R\right)
\end{equation}
<br>
4. Montrer que l'on peut écrire <br>
\begin{equation}
    F(\phi) = \sum_{k=1}^{N_{max}} k\zeta_k
\end{equation}
où les $\zeta_k$ sont des variables aléatoires indépendantes de loi de Poisson de paramètre $\eta_k$, que l'on calculera. <br>
**Réponse:** <br>
\begin{align*}
    F(\phi) =\,& \sum_{k=1}^{N_{max}} F(A_k) \\
            =\,& \sum_{k=1}^{N_{max}} \sum_{x\in A_k} N(x) \\
            =\,& \sum_{k=1}^{N_{max}} k\zeta_k,
\end{align*}
où $\zeta_k$ est le nombre d'utilisateurs dans $A_k$ suivant la loi de poissons, dont le parametre
\begin{align*}
    \eta_k =\,& \int_{0}^{2\pi} \int_{R_{k,min}}^{R_{k,max}} p\lambda r dr d \theta \\
           =\,& p\lambda\pi \left( R_{k,max}^2 - R_{k,min}^2 \right).
\end{align*}
<br>
5. Créer les vecteurs correspondants aux lois des variables aléatoires $\zeta_k$. <br>
**Réponse:** <br>
Si les $\eta_k$ supérieur à $1$ pour tous les $k$, on tronque pour $\zeta_k$ de $1$ à $3\eta_k$. 
Donc on représente pour $\zeta_k$ par le vecteur $\mathit{v}_k$, où
\begin{equation}
    \mathit{v}_{k,i} = e^{-\eta_k}\frac{\eta_k^{i}}{i!}.
\end{equation}
Sinon, on cherche par énumération afin de trouver un seuil $\delta$ tel que $\mathbb{P}\left(\zeta_k > \delta \right) < 10^{-4}$.
<br>
6. Construire les vecteurs représentant les lois des vecteurs $k\zeta_k$. <br>
**Réponse:** <br>
\begin{align*}
    \mathbb{P}\left( k\zeta_k = l\right) =\,& \mathbb{P}\left( \zeta_k = \frac{l}{k}\right) \\
                                         =\,& \begin{cases}
                                                   e^{-\eta_k}\frac{\eta_k^{m}}{m!}& l=m\cdot k,\,m\in \mathbb{N}
                                                                                                    \bigcap \left[1,\,3\eta_k\right] \\
                                                   0& sinon.
                                              \end{cases}
\end{align*}
Donc on construit pour $k\zeta_k$ le vecteur $\mathit{w}_k$, où
\begin{align*}
    \mathit{w}_{k,i} = \mathbb{P}\left( k\zeta_k = i\right).
\end{align*}
<br>
7. Calculer le vecteur représentant la loi de $F$. <br>
**Réponse:** <br>
\begin{align*}
    \mathbb{P}\left(F = f\right) =\,& \sum_{\sum f_k=f} \prod_{k=1}^{N_{max}}\mathit{P}\left( k\zeta_k=f_k\right) \\
                                 =\,& \sum_{f_1+f_{-1}=f} \mathit{P}\left(\zeta_1=f_1\right) 
                                      \sum_{f_{2}+f_{-2}=f_{-1}} \mathit{P}\left(2\zeta_2=f_2\right) 
                                      \sum ...
                                      \sum_{f_{N_{max}-1}+f_{N_{max}}=f_{-(N_{max}-2)}} 
                                          \mathit{P}\left(\left(N_{max}-1\right)\zeta_{N_{max}-1}=f_{N_{max}-1}\right) 
                                          \mathit{P}\left(N_{max}\zeta_{N_{max}}=f_{N_{max}}\right) \\
                                 =\,& \mathit{w}_1 * \left( \mathit{w}_2 * ...\mathit{w}_{N_{max}-2} * \left(
                                      \left(\mathit{w}_{N_{max}-1} * \mathit{w}_{N_{max}} \left[f_{-(N_{max}-2)} \right] \right)
                                      \left[f_{-(N_{max}-3)} \right]\right)...\right) \left[f \right],
\end{align*}
où $f_{-x}=f-\sum_{k=1}^x f_k$.
<br>
8. Trouver S tel que $p_0 < 0,01$. <br>
**Réponse:** <br>
\begin{equation}
    S^* = \min \lbrace S; \sum_{f=1}^S \mathbb{P}\left(F=f \right) > 0.99 \rbrace.
\end{equation}


## Analyse numérique 

In [1]:
def RBound(k, C=200, W=250, K=1e6,gamma=2.8):    
    return (K/(2**(C/(k*W))-1))**(1/gamma)

def Eta(RDown, RUp, lam=0.01, p=0.01):
    import math
    return p*lam*math.pi*(RUp**2-RDown**2)

def PoissonTroncate(param):
    import math
    ps = []
    if param >= 1:
        for k in range(int(3*param)):
            ps.append(math.e**(-param)*(param**(k+1)/math.factorial(k+1)))
    else:
        k = 1
        p = math.e**(-param)*(param**k/math.factorial(k))
        while p > 1e-4:            
            ps.append(p)
            k += 1
            p = math.e**(-param)*(param**k/math.factorial(k))
    return ps

def KZeta_k(v, k):
    import numpy as np
    length = len(v)
    eta = int(length/3)
    w = np.zeros(length*k)
    j = 0
    for i in range(length*k):
        if (i+1) % k == 0:
            w[i] = v[j]
            j += 1
    return w

In [2]:
C = 200
W = 250
K = 1e6
gamma = 2.8
R = 300
lam = 0.01
SNRMin = 0.1
p = 0.01
RCurrent = 0
slot = 1
As = []
vs = []
ws = []
while RCurrent <= R:
    RUp = RBound(k=slot)
    eta = Eta(RDown=RCurrent, RUp=min(RUp, R))
    As.append([slot, RCurrent, min(RUp, R), eta])
    RCurrent = RUp
    slot += 1
for A in As:
    eta = A[-1]
    k = int(A[0])
    v = PoissonTroncate(eta)
    vs.append(v)
    ws.append(KZeta_k(v, k))

Pour les $\eta_k$:

In [3]:
for A in As:
    print("k=%d Anneau(%.3f, %.3f) de paramètre %.3f" % (A[0], A[1], A[2], A[3]))
    print()

k=1 Anneau(0.000, 154.643) de paramètre 7.513

k=2 Anneau(154.643, 208.847) de paramètre 6.190

k=3 Anneau(208.847, 245.562) de paramètre 5.241

k=4 Anneau(245.562, 274.449) de paramètre 4.719

k=5 Anneau(274.449, 298.722) de paramètre 4.371

k=6 Anneau(298.722, 300.000) de paramètre 0.240



Pour les vecteur $\mathit{v}_k$:

In [4]:
for v in vs:
    print("Le vecteur représentant pour zeta_%d" % (vs.index(v)+1))
    for k in range(len(v)):
        print("Nombre d'utilisateur = %d a pour probabilité %.5f" % (k+1, v[k]))
    print()

Le vecteur représentant pour zeta_1
Nombre d'utilisateur = 1 a pour probabilité 0.00410
Nombre d'utilisateur = 2 a pour probabilité 0.01541
Nombre d'utilisateur = 3 a pour probabilité 0.03859
Nombre d'utilisateur = 4 a pour probabilité 0.07248
Nombre d'utilisateur = 5 a pour probabilité 0.10890
Nombre d'utilisateur = 6 a pour probabilité 0.13636
Nombre d'utilisateur = 7 a pour probabilité 0.14636
Nombre d'utilisateur = 8 a pour probabilité 0.13745
Nombre d'utilisateur = 9 a pour probabilité 0.11474
Nombre d'utilisateur = 10 a pour probabilité 0.08620
Nombre d'utilisateur = 11 a pour probabilité 0.05887
Nombre d'utilisateur = 12 a pour probabilité 0.03686
Nombre d'utilisateur = 13 a pour probabilité 0.02130
Nombre d'utilisateur = 14 a pour probabilité 0.01143
Nombre d'utilisateur = 15 a pour probabilité 0.00573
Nombre d'utilisateur = 16 a pour probabilité 0.00269
Nombre d'utilisateur = 17 a pour probabilité 0.00119
Nombre d'utilisateur = 18 a pour probabilité 0.00050
Nombre d'utilisateu

Pour les vecteurs $\mathit{w}_k$:

In [5]:
for w in ws:
    print("Le vecteur représentant pour %dzeta_%d" % ((ws.index(w)+1), (ws.index(w))+1))
    for k in range(len(w)):
        print("Nombre d'utilisateur = %d a pour probabilité %.5f" % (k+1, w[k]))
    print()

Le vecteur représentant pour 1zeta_1
Nombre d'utilisateur = 1 a pour probabilité 0.00410
Nombre d'utilisateur = 2 a pour probabilité 0.01541
Nombre d'utilisateur = 3 a pour probabilité 0.03859
Nombre d'utilisateur = 4 a pour probabilité 0.07248
Nombre d'utilisateur = 5 a pour probabilité 0.10890
Nombre d'utilisateur = 6 a pour probabilité 0.13636
Nombre d'utilisateur = 7 a pour probabilité 0.14636
Nombre d'utilisateur = 8 a pour probabilité 0.13745
Nombre d'utilisateur = 9 a pour probabilité 0.11474
Nombre d'utilisateur = 10 a pour probabilité 0.08620
Nombre d'utilisateur = 11 a pour probabilité 0.05887
Nombre d'utilisateur = 12 a pour probabilité 0.03686
Nombre d'utilisateur = 13 a pour probabilité 0.02130
Nombre d'utilisateur = 14 a pour probabilité 0.01143
Nombre d'utilisateur = 15 a pour probabilité 0.00573
Nombre d'utilisateur = 16 a pour probabilité 0.00269
Nombre d'utilisateur = 17 a pour probabilité 0.00119
Nombre d'utilisateur = 18 a pour probabilité 0.00050
Nombre d'utilisate

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  
